In [1]:
TRAIN = False
data_dir = '/scratch/es5223/medsynth/data_syntheticCT/TBX11K_orig_classification_splits/'
TYPE='proj'
TRIAL = 0

In [2]:
import os
import re
import cv2
import PIL
import sys
import json
import time
import timm
import math
import copy
import torch
import pickle
import logging
import fnmatch
import argparse
import itertools
import torchvision
from util import *
import numpy as np
%matplotlib inline
import pandas as pd
from apex import amp
import seaborn as sns
import albumentations
import torch.nn as nn
from PIL import Image
from pathlib import Path
from copy import deepcopy
import scikitplot as skplt
from sklearn import metrics
import torch.optim as optim
from timm import create_model
from datetime import datetime
from timm.data.loader import *
from torchvision import models
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.utils.data as data
from cutmix.cutmix import CutMix
from torch.autograd import Variable
from tqdm import tqdm, tqdm_notebook
from torch.optim import lr_scheduler
#from pytorch_metric_learning import loss
import torch.utils.model_zoo as model_zoo
from timm.models.layers.activations import *
from timm.utils import accuracy, AverageMeter
%config InlineBackend.figure_format = 'retina'
from apex.parallel import convert_syncbn_model
from timm.utils import ApexScaler, NativeScaler
from cutmix.utils import CutMixCrossEntropyLoss
from timm.models.registry import register_model
from collections import OrderedDict, defaultdict
from torch.utils.tensorboard import SummaryWriter
from warmup_scheduler import GradualWarmupScheduler
from timm.models.resnet import resnet26d, resnet50d
from torchvision import transforms, models, datasets
from timm.models.helpers import build_model_with_cfg
from torch.utils.data.sampler import SubsetRandomSampler
from randaugment import RandAugment, ImageNetPolicy, Cutout
from apex.parallel import DistributedDataParallel as ApexDDP
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy
from timm.models.efficientnet import efficientnet_b0, efficientnet_b1, efficientnet_b2, efficientnet_b3
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score, roc_curve, auc, roc_auc_score
#from timm.data import Dataset, DatasetTar, RealLabelsImagenet, create_loader, Mixup, FastCollateMixup, AugMixDataset

In [3]:
torch.backends.cudnn.benchmark = True

In [4]:
def load_image_joint(ct_path):
    ct_array = np.load(ct_path)
    tmp = ct_path.replace(data_dir,
                    '/scratch/es5223/medsynth/data/TBX11K_classification_splits/')
    tmp = tmp.replace('CT_','')
    tmp = tmp.replace('.npy','.png')
    if '.jpg.png' in tmp:
        tmp = tmp.replace('.png','')
    if not os.path.isfile(tmp):
        tmp = tmp.replace('.png','.jpg')
    if not os.path.isfile(tmp):
        print('ct_path ', ct_path)
        print('x_path ', tmp)

    x_img = cv2.imread(tmp, cv2.IMREAD_GRAYSCALE) * 1/256.
    x_image = cv2.resize(x_img,dsize=(224,224))
    # c_img = cv2.resize(ct_array[63,:,:],dsize=(224,224))
    # c_image = ct_array[32:96,:,:].transpose(2, 1, 0) #.convert('RGB')
    c_image = ct_array[:,:,:].transpose(2, 1, 0) #.convert('RGB')
    return [np.expand_dims(c_image, axis=0), np.expand_dims(x_image, axis=0)] #img_concat

def is_valid_file(path):
    return True

In [ ]:
device = torch.device("cuda:0")
batch_size = 10
num_epochs = 500
lr = 0.01
beta = 1
step_size = 130
img_size_c = 128
img_size = 224
test_size = int((256 / 224) * img_size)

mean = [0.485] 
std = [0.229]
num_workers = 0
# Define your transforms for the training and testing sets
data_transforms_x = {
    'train': transforms.Compose([
        #transforms.Resize(224),
        #transforms.ToTensor(),
        transforms.Normalize(0., 1.),
        transforms.RandomErasing()
    ]),
    'val': transforms.Compose([
        #transforms.Resize(224),
        #transforms.ToTensor(),
        transforms.Normalize(0., 1.)
        #transforms.RandomErasing()
    ]),
    'test': transforms.Compose([
        #transforms.Resize(224),
        #transforms.ToTensor(),
        transforms.Normalize(0., 1.)
        #transforms.RandomErasing()
    ])
}


data_transforms_ct = {
    'train': transforms.Compose([
        #transforms.RandomRotation(30),
        #transforms.Resize(img_size),
        #transforms.ToTensor(),
        transforms.Normalize(0., 1.),
        #transforms.RandomErasing(),
    ]),
    'val': transforms.Compose([
        #transforms.Resize(img_size),
        #transforms.ToTensor(),
        transforms.Normalize(0., 1.),
    ]),
    'test': transforms.Compose([
        #transforms.Resize(img_size),
        #transforms.ToTensor(),
        transforms.Normalize(0., 1.),
    ])
}


# Load the datasets with ImageFolder
print("data_dir ", data_dir)
# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), 
                                          #data_transforms_ct[x],
                                          loader=load_image_joint,
                                          is_valid_file=is_valid_file) for x in ['train', 'val', 'test']}

class_names = image_datasets['train'].classes
num_classes = len(class_names)
data_loader = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=num_workers, pin_memory = True)
              for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

print(class_names)
print(dataset_sizes)
print(device)

### we get the class_to_index in the data_Set but what we really need is the cat_to_names  so we will create
_ = image_datasets['val'].class_to_idx
cat_to_name = {_[i]: i for i in list(_.keys())}
print(cat_to_name)
    
# Run this to test the data loader
images, labels = next(iter(data_loader['val']))
print("len(images) ", len(images))
print("images[0].shape ", images[0].shape)

In [ ]:
print("len(images) ", len(images))
print("images[0].shape ", images[0].shape)

ct_data = images[0]
x_data = images[1]

ct_data_transformed = data_transforms_ct['train'](ct_data)
print(ct_data_transformed.shape)

x_data_transformed = data_transforms_ct['train'](x_data)
print(x_data_transformed.shape)

In [ ]:
class CNN3D(torch.nn.Module):
    def __init__(self, num_classes):
        super(CNN3D, self).__init__()
        """Build a 3D convolutional neural network model."""
        self.relu = torch.nn.ReLU()
        self.maxPool = torch.nn.MaxPool3d(kernel_size=2)


        self.conv1 = torch.nn.Conv3d(1, 64, kernel_size=(3,3,3))
        self.batchNorm1 = torch.nn.BatchNorm3d(num_features=64)


        self.conv2 = torch.nn.Conv3d(64, 64, kernel_size=(3,3,3))
        self.batchNorm2 = torch.nn.BatchNorm3d(num_features=64)

        self.conv3 = torch.nn.Conv3d(64, 128, kernel_size=(3,3,3))
        self.batchNorm3 = torch.nn.BatchNorm3d(num_features=128)

        self.conv4 = torch.nn.Conv3d(128, 256, kernel_size=(3,3,3))
        self.batchNorm4 = torch.nn.BatchNorm3d(num_features=256)


        self.avgPool = torch.nn.AvgPool3d(kernel_size=(6,6,6))
        self.fc1 = torch.nn.Linear(256, 512)
        self.dropout = torch.nn.Dropout3d(0.3)
        #self.fc2 = torch.nn.Linear(512, 1)
        self.sigmoid = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxPool(x)
        x = self.batchNorm1(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxPool(x)
        x = self.batchNorm2(x)

        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxPool(x)
        x = self.batchNorm3(x)

        x = self.conv4(x)
        x = self.relu(x)
        x = self.maxPool(x)
        x = self.batchNorm4(x)

        x = self.avgPool(x).squeeze()
        x = self.fc1(x)

        x = self.dropout(x)
        x = self.fc2(x)
        x = self.sigmoid(x)#.squeeze()
        return x
    

In [ ]:
data_ct = torch.randn((batch_size, 1, 128, 128, 128))
data_x = torch.randn((batch_size, 1, 224, 224))

output = model(data_ct, data_x)
print(output.shape)

In [ ]:
# Create classifier
for param in model.parameters():
    param.requires_grad = True

criterion = LabelSmoothingCrossEntropy()
#criterion = CutMixCrossEntropyLoss(True)
#criterion = nn.CrossEntropyLoss()
#optimizer = Nadam(model.parameters(), lr=0.001)
#optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)


optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)
#lr = lambda x: (((1 + math.cos(x * math.pi / num_epochs)) / 2) ** 1) * 0.9
#scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=5, after_scheduler=scheduler)
#model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
#loss_scaler = ApexScaler()
#show our model architechture and send to GPU
model.to(device)
model.model_x.to(device)
model.model_ct.to(device)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count = count_parameters(model)
print("The number of parameters of the model is:", count)

In [10]:
#https://github.com/clovaai/CutMix-PyTorch
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [11]:
# # Create classifier
# for param in model.model_x.parameters():
#     param.requires_grad = False

# for param in model.model_ct.parameters():
#     param.requires_grad = False

# model_x.head.weight
# model_x.head.bias
# model_ct.fc2.weight
# model_ct.fc2.bias
# fc1.weight
# fc1.bias


params_to_optimize = list(model.model_x.head.parameters()) + list(model.model_ct.fc2.parameters()) + list(model.fc1.parameters())
optimizer = optim.SGD(params_to_optimize, lr=lr, momentum=0.9, nesterov=True, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)
#lr = lambda x: (((1 + math.cos(x * math.pi / num_epochs)) / 2) ** 1) * 0.9
#scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=5, after_scheduler=scheduler)

# model.fc1

# compare two models

In [13]:
import glob 

data_transforms_ct_L = {
    'train': transforms.Compose([
        #transforms.RandomRotation(30),
        #transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize(0., 1.),
        #transforms.RandomErasing(),
    ]),
    'val': transforms.Compose([
        #transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize(0., 1.),
    ]),
    'test': transforms.Compose([
        #transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize(0., 1.),
    ])
}


def load_image_joint_print(ct_path):
    ct_array = np.load(ct_path)
    tmp = ct_path.replace(data_dir,
                    '/scratch/es5223/medsynth/data/TBX11K_classification_splits/')
    tmp = tmp.replace('CT_','')
    tmp = tmp.replace('.npy','.png')
    if '.jpg.png' in tmp:
        tmp = tmp.replace('.png','')
    if not os.path.isfile(tmp):
        tmp = tmp.replace('.png','.jpg')
    if not os.path.isfile(tmp):
        print('ct_path ', ct_path)
        print('x_path ', tmp)

    x_img = cv2.imread(tmp, cv2.IMREAD_GRAYSCALE) * 1/256.
    x_image = cv2.resize(x_img,dsize=(224,224))
    # c_img = cv2.resize(ct_array[63,:,:],dsize=(224,224))
    # c_image = ct_array[32:96,:,:].transpose(2, 1, 0) #.convert('RGB')
    c_image = ct_array[:,:,:].transpose(2, 1, 0) #.convert('RGB')
    c_image_t = c_image #np.expand_dims(c_image, axis=0)
    c_image_t = data_transforms_ct_L['test'](c_image_t)
    return [c_image_t, np.expand_dims(x_image, axis=0)] #img_concat

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    Source: https://gist.github.com/andrewjong/6b02ff237533b3b2c554701fb53d5c4d
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

    
test_dataset = ImageFolderWithPaths(os.path.join(data_dir, 'test'), 
                                          #data_transforms_ct['test'],
                                          loader=load_image_joint_print,
                                          is_valid_file=is_valid_file)

test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                             shuffle=False, num_workers=0, pin_memory = True)

In [14]:
# load x model
model_X_only = create_model('vit_small_patch16_224', pretrained=True, in_chans=1, num_classes=num_classes)

In [ ]:
def compute_validate_meterX(model, val_loader, CHECK_POINT_PATH): # best_model_path,
    since = time.time()
    try:
        checkpoint = torch.load(CHECK_POINT_PATH)
        print("checkpoint loaded")
    except:
        checkpoint = None
        print("checkpoint not found")

    def load_model(best_model_path):                                
        model.load_state_dict(checkpoint['model_state_dict'])
        best_model_wts = copy.deepcopy(model.state_dict())
        best_loss = checkpoint['best_val_loss']
        best_acc = checkpoint['best_val_accuracy']
        
    device = torch.device("cuda:0")
    load_model(CHECK_POINT_PATH)
    model.to(device)
    model.eval()
    pred_y = list()
    test_y = list()
    probas_y = list()
    with torch.no_grad():
        for data, target,paths in val_loader:
            #print("data[0].shape ", data[0].shape)
            #print("data[1].shape ", data[1].shape)
            #print("target.shape ", target.shape)
            #data, target = data[0].to(device), target.to(device)
            target = target.to(device)
            #ct_data = data[0]
            x_data = data[1]

            #ct_data_transformed = data_transforms_ct[phase](ct_data)
            x_data_transformed = data_transforms_x['test'](x_data)

            #ct_data_transformed = ct_data_transformed.float().to(device)
            x_data_transformed = x_data_transformed.float().to(device)
                    
            data, target = Variable(x_data_transformed), Variable(target)
            output = model(data.float())
            probas_y.extend(output.data.cpu().numpy().tolist())
            pred_y.extend(output.data.cpu().max(1, keepdim=True)[1].numpy().flatten().tolist())
            test_y.extend(target.data.cpu().numpy().flatten().tolist())
        # compute the confusion matrix
        confusion = confusion_matrix(test_y, pred_y)
        # plot the confusion matrix
        plot_labels = ['HEALTH', 'SICK', 'TB']
        #plot_confusion_matrix(confusion, plot_labels)
        #plot_confusion_matrix(confusion, classes=val_loader.dataset.classes,title='Confusion matrix')
        # print Recall, Precision, F1-score, Accuracy
        report = classification_report(test_y, pred_y, digits=4)
        print(report)
        #plt_roc(test_y, probas_y)
    time_elapsed = time.time() - since
    print('Inference completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return test_y, pred_y

test_y_X, pred_y_X = compute_validate_meterX(model_X_only, test_data_loader, 'models_trials/X_orig_VT_TRIAL_0.pth')

In [ ]:
test_y_X0, pred_y_X0 = compute_validate_meterX(model_X_only, test_data_loader, 'models_trials/X_orig_VT_TRIAL_0.pth')
test_y_X1, pred_y_X1 = compute_validate_meterX(model_X_only, test_data_loader, 'models_trials/X_orig_VT_TRIAL_1.pth')
test_y_X2, pred_y_X2 = compute_validate_meterX(model_X_only, test_data_loader, 'models_trials/X_orig_VT_TRIAL_2.pth')

In [10]:
# get CT only results
from model_cnn import CNN3D

# load ct model
model_CT_only = CNN3D(num_classes=3)

In [ ]:
print(data_dir)

def load_image(path):
    ct_array = np.load(path)
    # image = Image.fromarray(ct_array[63,:,:])#.convert('RGB')
    image = ct_array[:,:,:].transpose(2, 1, 0) #.convert('RGB')
    return image

def is_valid_file(path):
    return True

def compute_validate_meterCT(model, val_loader, CHECK_POINT_PATH): # best_model_path,
    since = time.time()
    try:
        checkpoint = torch.load(CHECK_POINT_PATH)
        print("checkpoint loaded")
    except:
        checkpoint = None
        print("checkpoint not found")

    def load_model(best_model_path):                                
        model.load_state_dict(checkpoint['model_state_dict'])
        best_model_wts = copy.deepcopy(model.state_dict())
        best_loss = checkpoint['best_val_loss']
        best_acc = checkpoint['best_val_accuracy']
        
    device = torch.device("cuda:0")
    load_model(CHECK_POINT_PATH)
    model.to(device)
    model.eval()
    pred_y = list()
    test_y = list()
    probas_y = list()
    with torch.no_grad():
        for data, target,paths in val_loader:
            target_orig = target
            target = torch.nn.functional.one_hot(target, num_classes=3)
            target = target.float()
            target = target.to(device)
            ct_data = data[0].unsqueeze(1)

            #ct_data_transformed = data_transforms_ct['test'](ct_data)
            #@ct_data_transformed = ct_data_transformed.float().to(device)
            data = ct_data#ct_data_transformed
            data=data.to(device)
#         for data, target in val_loader:
#             # print("paths ", paths)
#             target_orig = target
#             target = torch.nn.functional.one_hot(target, num_classes=3)
#             target = target.float()
            
            data, target = Variable(data), Variable(target)

            output = model(data)
            probas_y.extend(output.data.cpu().numpy().tolist())
            pred_y.extend(output.data.cpu().max(1, keepdim=True)[1].numpy().flatten().tolist())
            test_y.extend(target_orig.data.cpu().numpy().flatten().tolist())
            
        # compute the confusion matrix
        confusion = confusion_matrix(test_y, pred_y)
        # plot the confusion matrix
        plot_labels = ['HEALTH', 'SICK', 'TB']
        #plot_confusion_matrix(confusion, plot_labels)
        #plot_confusion_matrix(confusion, classes=val_loader.dataset.classes,title='Confusion matrix')
        # print Recall, Precision, F1-score, Accuracy
        report = classification_report(test_y, pred_y, digits=4)
        print(report)
        #plt_roc(test_y, probas_y)
    time_elapsed = time.time() - since
    print('Inference completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return test_y, pred_y
test_y_CT, pred_y_CT = compute_validate_meterCT(model_CT_only, test_data_loader, 'models_trials/3D_CNN_bce_allSlice+proj0.pth')


In [ ]:
test_y_CT0, pred_y_CT0 = compute_validate_meterCT(model_CT_only, test_data_loader, 'models_trials/3D_CNN_bce_allSlice+proj0.pth')
test_y_CT1, pred_y_CT1 = compute_validate_meterCT(model_CT_only, test_data_loader, 'models_trials/3D_CNN_bce_allSlice+proj1.pth')
test_y_CT2, pred_y_CT2 = compute_validate_meterCT(model_CT_only, test_data_loader, 'models_trials/3D_CNN_bce_allSlice+proj2.pth')